# AWSClient

In [ ]:
from   collections import defaultdict
import hashlib
import json
import re
import time

import pandas

from   datalabs.access.aws import AWSClient

## Extractor

In [ ]:
sheet = pandas.read_excel("snomed_cpt_map_fsn_descr_20230131.xlsx")

In [ ]:
sheet.columns.values

## Transformer

In [ ]:
snomed = sheet.loc[:, ["Concept Id", "FSN", "Map Category", "CPT Code", "CPT Descriptor", "Map Id"]]

snomed.loc[:, "Concept Id"] = snomed["Concept Id"].fillna(method='ffill').astype('int').astype('str')
snomed.loc[:, "FSN"] = snomed["FSN"].fillna(method='ffill')
snomed.loc[:, "FSN"] = snomed["FSN"].astype('str')
snomed.loc[snomed["FSN"] == "nan", "FSN"] = ""

snomed.loc[:, "CPT Descriptor"] = snomed["CPT Descriptor"].astype('str')
snomed.loc[snomed["CPT Descriptor"] == "nan", "CPT Descriptor"] = ""

snomed

In [ ]:
snomed.loc[:, "pk"] = "CONCEPT:" + snomed["Concept Id"]

snomed.loc[snomed["Map Category"] == "Unmappable", "sk"] = "UNMAPPABLE:" + snomed.loc[snomed["Map Category"] == "Unmappable", "Map Id"].astype('str')
snomed.loc[~(snomed["Map Category"] == "Unmappable"), "sk"] = "CPT:" + snomed.loc[~(snomed["Map Category"] == "CPT"), "CPT Code"].astype(str)
snomed = snomed.drop_duplicates(subset=("pk", "sk"))

mappings = snomed.loc[:, ["sk", "pk", "FSN", "Map Category", "CPT Descriptor"]].rename(
    columns={
        "FSN": "snomed_descriptor",
        "Map Category": "map_category",
        "CPT Descriptor": "cpt_descriptor",
    }
)

mappings

In [ ]:
mappings["keyword"] = mappings.snomed_descriptor.apply(lambda x: re.sub(r'[^\w ]+', '', x)).str.lower().str.split()
keyword_map = mappings.loc[:, ["pk", "sk", "keyword"]].explode("keyword").reset_index(drop=True).drop_duplicates()
keyword_map = keyword_map.groupby(["keyword", "pk"]).sk.agg(lambda items: [x for x in items if x.startswith("CPT:")])

mappings = mappings.drop(columns="keyword")

# for (keyword, pk), mapping in keyword_groups:
#     print(pk)
#     print(keys.head(5), end="\n\n")
keyword_map

In [ ]:
keyword_map[("zygomatic", "CONCEPT:257904000")]

In [ ]:
# keyword_map[keyword_map.pk == "CONCEPT:609038002"]

In [ ]:
items = []

for index in range(len(mappings)):
    row = mappings.iloc[index]

    items.append(row.to_dict())

# items += [dict(pk=f"{row.pk}:{row.sk}", sk=f"KEYWORD:{row.keyword}") for index, row in keyword_map.iterrows() if row.sk.startswith("CPT:")]

# items

for keys, codes in keyword_map.items():
    items.append(dict(pk=keys[1], sk=f"KEYWORD:{keys[0]}", codes=codes))

items

In [ ]:
print(len(items))
items[-10:]

In [ ]:
# mappings.loc[(mappings.pk == "CONCEPT:609038002")]
descriptors = mappings.loc[(mappings.pk == "CONCEPT:609038002") & (mappings.sk == "CPT:88160"), ("cpt_descriptor")]
list(descriptors)

## Loader

In [ ]:
with open("snomed_cpt_mappings.json", "w") as file:
    file.write(json.dumps(items))

In [ ]:
with open("snomed_cpt_mappings.json", "r") as file:
    items = json.loads(file.read())

In [ ]:
# snomed_table = snomed
# snomed_table["md5"] = snomed[["pk", "sk", "snomed_descriptor", "map_category", "cpt_descriptor"]].apply(lambda row: hashlib.md5(str(row.values).encode('utf-8')).hexdigest(), axis=1)
# snomed_table

# for item in items:
#     item.pop("md5")
items

In [ ]:
hashed_items = items

for item in items:
    if item["sk"].startswith("UNMAPPABLE:") or item["sk"].startswith("CPT:"):
        md5 = hashlib.md5(json.dumps(item, sort_keys=True).encode('utf-8')).hexdigest()
        hashed_items.append(dict(pk=f'{item["pk"]}:{item["sk"]}', sk=f"MD5:{md5}"))

hashed_items

In [ ]:
print(len(hashed_items))
hashed_items[-120000:]

In [ ]:
# Swap after reload
# with AWSClient("s3") as db:
#     table = db.resource.Table('CPT-API-snomed-sbx')
#     dir(table)
import boto3
table = boto3.resource("dynamodb").Table('CPT-API-snomed-sbx')

from bisect import bisect_left

In [ ]:
json_items = sorted([json.dumps(item, sort_keys=True) for item in hashed_items])
unique_items = set(json.dumps(item, sort_keys=True) for item in hashed_items)

duplicate_json_items = []

for item in unique_items:
    index = bisect_left(json_items, item)
    if index < (len(json_items)-1) and json_items[index+1] == item:
        duplicate_json_items.append(item)

duplicate_json_items

## Load Sandbox

In [ ]:
start_time = time.perf_counter()

with boto3.resource("dynamodb").Table('CPT-API-snomed-sbx').batch_writer():
    for item in hashed_items:
        batch.put_item(Item=item)

time.perf_counter() - start_time

## Load Dev

In [ ]:
start_time = time.perf_counter()

with AWSClient("dynamodb").resource as dynamodb:
    table = dynamodb.Table('CPT-API-snomed-dev')

    with table.batch_writer() as batch:
        for item in hashed_items:
            batch.put_item(Item=item)

time.perf_counter() - start_time

In [ ]:
start_time = time.perf_counter()

with AWSClient("dynamodb") as dynamodb:
    results = paginate(dynamodb, "SELECT * FROM \"CPT-API-snomed-sbx\".\"SearchIndex\" WHERE begins_with(\"sk\", 'MD5:')")

    results = list(results)

print(time.perf_counter() - start_time)
results

In [ ]:
current_hashes_columns = defaultdict(list)

for result in results:
    for key, value in result.items():
        current_hashes_columns[key].append(value["S"])

current_hashes = pandas.DataFrame(current_hashes_columns)
current_hashes

In [ ]:
incoming_hashes = current_hashes.copy()

incoming_hashes = pandas.concat((
    incoming_hashes,
    pandas.DataFrame(
        dict(
            sk=["MD5:1234567890abcdefghijklmnopqrstuv", "MD5:abcdefghijklmnopqurstvwxyz123456", "MD5:d4de199db48813dc0e4133a480aaf6b8", "MD5:388936416f61c9255b4e4842764faf66"],
            pk=['CONCEPT:123456789:CPT:12345', 'CONCEPT:987654321:CPT:54321', incoming_hashes.pk[incoming_hashes.sk == "MD5:133a480aaf6b8d4de199db48813dc0e4"].iloc[0], incoming_hashes.pk[incoming_hashes.sk == "MD5:4e4842764faf66388936416f61c9255b"].iloc[0]]
        )
    )
))

incoming_hashes = incoming_hashes[~incoming_hashes.sk.isin(("MD5:133a480aaf6b8d4de199db48813dc0e4", "MD5:4e4842764faf66388936416f61c9255b", "MD5:adfec4b4db82f70b2fc7d4c6f261cc7b", "MD5:d176d88ac597ec61d825ff50bd02e02f"))]

incoming_hashes

In [ ]:
start_time = time.perf_counter()

deleted_hashes = current_hashes[~current_hashes.pk.isin(incoming_hashes_df.pk)]

new_or_updated_hashes = incoming_hashes[~incoming_hashes.sk.isin(current_hashes.sk)]

new_hashes = new_or_updated_hashes[~new_or_updated_hashes.pk.isin(current_hashes.pk)]

updated_hashes = new_or_updated_hashes[new_or_updated_hashes.pk.isin(current_hashes.pk)]

print(time.perf_counter() - start_time)

In [ ]:
deleted_hashes

In [ ]:
new_hashes

In [ ]:
updated_hashes

## Get Deleted Keywords for Hashes

In [ ]:
start_time = time.perf_counter()
deleted_keywords = []

with AWSClient("dynamodb") as dynamodb:
    for pk in deleted_hashes.pk:
        results = paginate(dynamodb, f"SELECT * FROM \"CPT-API-snomed-sbx\" WHERE pk = '{pk}' AND begins_with(\"sk\", 'KEYWORD:')")

    deleted_keywords.append(list(results))

print(time.perf_counter() - start_time)
deleted_keywords

## /snomed/map/cpt/{concept}

In [ ]:
def generate_map(items):
    mappings = []

    for item in items:
        mapping = {key:value['S'] for key, value in item.items()}

        mapping.pop("pk")
        mapping["cpt_code"] = mapping.pop("sk").replace("CPT:", "")

        mappings.append(mapping)

    return mappings

In [ ]:
concept = "306683007"

with AWSClient("dynamodb") as db:
    results = db.execute_statement(
        Statement=f"SELECT * FROM \"CPT-API-snomed-sbx\" WHERE pk = 'CONCEPT:{concept}' AND begins_with(\"sk\", 'CPT:')"
    )

print(generate_map(results["Items"]))


## /snomed/maps/cpt

In [ ]:
def get_mapping_references_for_keyword(keyword, db):
    maps = None

    search_results = db.execute_statement(
        Statement=f"SELECT * FROM \"CPT-API-snomed-sbx\".\"SearchIndex\" WHERE sk = 'KEYWORD:{keyword}'"
    )

    return search_results["Items"]

In [ ]:
def get_mappings_from_references(keyword_items, db):
    mappings = defaultdict(list)

    for search_item in keyword_items:
        mapping = get_mapping_from_reference(search_item['pk']['S'], db)

        mappings[mapping["pk"]["S"]].append(mapping)

    return mappings

In [ ]:
# TODO: handle DynamoDB pagination

def get_all_mappings(db):
    mappings = defaultdict(list)

    results = db.execute_statement(
        Statement=f"SELECT * FROM \"CPT-API-snomed-sbx\" WHERE begins_with(\"sk\", 'CPT:')"
    )
    
    for item in results["Items"]:
        mappings[item["pk"]["S"]].append(item)

    return mappings

In [ ]:
def generate_map(mapping_items):
    mappings = []
    concept = None

    for item in mapping_items:
        mapping = {key:value['S'] for key, value in item.items()}

        if not concept:
            concept = mapping["pk"]
            snomed_descriptor = mapping["snomed_descriptor"]

        mappings.append(
            dict(
                code=mapping["sk"].replace("CPT:", ""),
                descriptor=mapping["cpt_descriptor"],
                category=mapping["map_category"]
            )
        )

    return dict(
        concept=concept,
        descriptor=snomed_descriptor,
        mappings=mappings
    )

In [ ]:
def get_mapping_from_reference(pksk, db):
    pk = pksk.rsplit(':', 2)[0]   # CONCEPT:...
    sk = pksk.split(':', 2)[2]    # CPT:...

    concept = pk.replace("CONCEPT:", "")

    results = db.execute_statement(
        Statement=f"SELECT * FROM \"CPT-API-snomed-sbx\" WHERE pk = '{pk}' AND sk = '{sk}'"
    )
    
    return results["Items"][0]

## Use Case 1: Get Mappings by Keyword

In [ ]:
# TODO: handle response pagination

keyword = "discharge"

with AWSClient("dynamodb") as db:
    mapping_references = get_mapping_references_for_keyword(keyword, db)
    
    mapping_set = get_mappings_from_references(mapping_references, db)

maps = [generate_map(mappings) for mappings in mapping_set.values()]
maps

In [ ]:
with AWSClient("dynamodb") as db:
    results = get_mapping_references_for_keyword("discharge", db)

results

## Use Case 2: Get All Mappings (Default)

In [ ]:
# TODO: handle response pagination

with AWSClient("dynamodb") as db:
    mapping_set = get_all_mappings(db)

maps = [generate_map(mappings) for mappings in mapping_set.values()]
maps

## Pagination

In [ ]:
def paginate(db, statement):
    results = db.execute_statement(Statement=statement, Limit=600000)

    for item in results["Items"]:
        yield item
    
    while "NextToken" in results:
        results = db.execute_statement(Statement=statement, Limit=600000, NextToken=results["NextToken"])
        
        for item in results["Items"]:
            yield item